### Import the needed package

In [21]:
# package for handing files
import os
# package for handling images
import numpy as np
import cv2
from PIL import Image
# package for handling json files
import json

### Global variable

In [22]:
path_main = "./../../../dataset_processed"
path_img = os.path.join(path_main, "fundus_image")
path_anot = os.path.join(path_main, "annotations")
path_mask = "./../../../dataset_used/mask_image"
classes = ["glaucoma", "non_glaucoma"]

### Get the annotation information

In [23]:
annots = {}
for class_type in classes:
    annots[class_type] = json.load(open([os.path.join(path_anot,
                                                        class_type,
                                                        file)
                                            for file in os.listdir(os.path.join(path_anot,
                                                                                class_type))
                                            if file.endswith(".json")][0]))

### Create the mask image

In [31]:
# loop for each classes (glaucoma, non_glaucoma)
for class_type, annotations in annots.items():
    # loop for every image in the class
    for annotation in annotations:
        # make sure the image have disc and cup annotation
        if len(annotation["annotation"]) < 2:
            continue
        # create a black image with the same size as the image
        b_mask = np.zeros((int(annotation["metadata"]["img_height"]),
                            int(annotation["metadata"]["img_width"])), dtype=np.uint8)
        
        # draw the white space (mask area)
        for polygon in sorted(annotation["annotation"], key=lambda x: x["label"], reverse=True):
            # define the color of the mask
            label = 255 if polygon["label"] == "disc" else 64
            # get points of the polygon
            points = [list(map(np.float32, point.split(","))) for point in polygon["points"].split(";")]
            # fill the polygon with the color
            cv2.fillPoly(b_mask, [np.array(points, dtype=np.int32)], color=label)
        # check the image already exist or not
        try:
            os.remove(os.path.join(path_mask, f"{annotation['metadata']['img_name'].split('.')[0]}.png"))
        except FileNotFoundError:
            pass
        # save the mask image
        Image.fromarray(b_mask).save(os.path.join(path_mask, f"{annotation['metadata']['img_name'].split('.')[0]}_mask.png"))